### 모델 구성과 실행을 분리

In [ ]:
import tensorflow as tf

In [ ]:
hello = tf.constant('Hello, Tensorflow')

In [ ]:
a = tf.constant(10)
b = tf.constant(32)
c = tf.add(a, b)

In [ ]:
sess = tf.Session()
print (sess.run(hello))
print (sess.run([a,b,c]))
sess.close()

In [ ]:
import tensorflow as tf

In [ ]:
x = tf.placeholder(tf.float32, [None, 3])
x_data = [[1,2,3],[4,5,6]]

In [ ]:
w = tf.Variable(tf.random_normal([3,2]))
b = tf.Variable(tf.random_normal([2,1]))

In [ ]:
expr = tf.matmul(x,w) + b

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
print("=== x_data ===")
print(x_data)
print("=== w ===")
print(sess.run(w))
print("=== b ===")
print(sess.run(b))
print(sess.run(tf.matmul(x,w), feed_dict={x:x_data}))
print("=== expr ===")
print(sess.run(expr, feed_dict={x:x_data}))
sess.close()

In [ ]:
x_data = [1,2,3]
y_data = [1,2,3]

In [ ]:
w = tf.Variable(tf.random_uniform([1], -1.0, 1.0))
b = tf.Variable(tf.random_uniform([1], -1.0, 1.0))

In [ ]:
x = tf.placeholder(tf.float32, name='x')
y = tf.placeholder(tf.float32, name='y')

In [ ]:
hypothesis = w*x + b

In [ ]:
cost = tf.reduce_mean(tf.square(hypothesis - y))

In [ ]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)

In [ ]:
train = optimizer.minimize(cost)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(300):
        _, cost_val = sess.run([train, cost], feed_dict={x:x_data, y:y_data})
        if step % 30 == 0:
            print (step, cost_val, sess.run(w), sess.run(b))
        
    print("\n=== Test ===")
    print("X: 5, Y:", sess.run(hypothesis, feed_dict={x: 5}))
    print("X: 2.5, Y:", sess.run(hypothesis, feed_dict={x: 2.5}))    

In [ ]:
# 털과 날개가 있는지 없는지에 따라, 포유류인지 조류인지 분류하는 신경망 모델을 만들어봅니다.
import tensorflow as tf
import numpy as np

In [ ]:
# [털, 날개]
# [0, 0] = [털X, 날개X] = 기타
# [0, 1] = [털X, 날개O] = 기타
# [1, 0] = [털O, 날개X] = 포유류
# [1, 1] = [털O, 날개O] = 조류
x_data = np.array([[0, 0], [1, 0], [1, 1], [0, 0], [0, 0], [0, 1]])

In [ ]:
# class 3개 [기타, 포유류, 조류]
# one-hot 형식
y_data = np.array([[1, 0, 0],[0, 1, 0],[0, 0, 1],[1, 0, 0],[1, 0, 0],[0, 0, 1]])

In [ ]:
x = tf.placeholder(tf.float32, [None, 2])
y = tf.placeholder(tf.float32, [None, 3])

In [ ]:
w1 = tf.Variable(tf.random_uniform([2,10], -1., 1.))
w2 = tf.Variable(tf.random_uniform([10,3], -1., 1.))
                 
b1 = tf.Variable(tf.zeros([10]))
b2 = tf.Variable(tf.zeros([3]))                 

In [ ]:
L1 = tf.matmul(x,w1) + b1
L1 = tf.nn.relu(L1)

In [ ]:
model = tf.matmul(L1, w2) + b2

In [ ]:
# 각 개별 결과에 대한 합을 구한 뒤 평균을 내는 방식을 사용합니다.
# 전체 합이 아닌, 개별 결과를 구한 뒤 평균을 내는 방식을 사용하기 위해 axis 옵션을 사용합니다.
# axis 옵션이 없으면 -1.09 처럼 총합인 스칼라값으로 출력됩니다.
#        Y         model         Y * tf.log(model)   reduce_sum(axis=1)
# 예) [[1 0 0]  [[0.1 0.7 0.2]  -> [[-1.0  0    0]  -> [-1.0, -0.09]
#     [0 1 0]]  [0.2 0.8 0.0]]     [ 0   -0.09 0]]
# Cross-Entropy : 예측값과 실제값 사이의 확률 분포 차이
# Y는 one-hot 이므로 1개만 1이기 때문에, model 값 중 하나만 선택 (model 원소들의 합 = 1) )
# 해당 model 값(확률 값)이 1에 가까울수록 log10(1)=0
# 해당 model 값(확률 값)이 0에 가까울수록 log10(x)=-inf

In [ ]:
# cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=model))
model = tf.nn.softmax(model)
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(model), axis=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train = optimizer.minimize(cost)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(100):
        sess.run(train, feed_dict={x:x_data, y:y_data})
        if (step + 1) % 10 == 0:
            print(step + 1, sess.run(cost, feed_dict={x: x_data, y: y_data}))
            
    prediction = tf.argmax(model, 1)
    target = tf.argmax(y, 1)
    print('예측값:', sess.run(prediction, feed_dict={x:x_data}))
    print('실제값:', sess.run(target, feed_dict={y:y_data}))
    is_correct = tf.equal(prediction, target)
    accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
    print('정확도:%.2f'% sess.run(accuracy * 100, feed_dict={x: x_data, y: y_data}) + '%')

In [1]:
# word2vec
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [3]:
sentences = ['I like cat',
             'I like dog',
             'I like animal',
             'dog cat animal',
             'gf cat dog like',
             'cat fish milk like',
             'dog fish hate milk like',
             'dog cat eye like',
             'I gf like',
             'gf I hate',
             'gf I movie book music like',
             'I game cartoon animation like',
             'cat dog hate',
             'dog cat like']

In [6]:
# 문장을 전부 합친 후 공백으로 단어들을 나누고 고유한 단어들로 리스트를 만듭니다.
word_sequence = " ".join(sentences).split()
word_sequence

['I',
 'like',
 'cat',
 'I',
 'like',
 'dog',
 'I',
 'like',
 'animal',
 'dog',
 'cat',
 'animal',
 'gf',
 'cat',
 'dog',
 'like',
 'cat',
 'fish',
 'milk',
 'like',
 'dog',
 'fish',
 'hate',
 'milk',
 'like',
 'dog',
 'cat',
 'eye',
 'like',
 'I',
 'gf',
 'like',
 'gf',
 'I',
 'hate',
 'gf',
 'I',
 'movie',
 'book',
 'music',
 'like',
 'I',
 'game',
 'cartoon',
 'animation',
 'like',
 'cat',
 'dog',
 'hate',
 'dog',
 'cat',
 'like']

In [7]:
# 유일한 단어만 선정
word_list = list(set(word_sequence))
word_list

['game',
 'like',
 'I',
 'gf',
 'fish',
 'milk',
 'dog',
 'animal',
 'cat',
 'eye',
 'cartoon',
 'hate',
 'movie',
 'music',
 'book',
 'animation']

In [8]:
word_dict = {w: i for i, w in enumerate(word_list)}
word_dict

{'I': 2,
 'animal': 7,
 'animation': 15,
 'book': 14,
 'cartoon': 10,
 'cat': 8,
 'dog': 6,
 'eye': 9,
 'fish': 4,
 'game': 0,
 'gf': 3,
 'hate': 11,
 'like': 1,
 'milk': 5,
 'movie': 12,
 'music': 13}

In [11]:
# 윈도우 사이즈를 1 로 하는 skip-gram 모델을 만듭니다.
# 예) 나 게임 만화 애니 좋다
#   -> ([나, 만화], 게임), ([게임, 애니], 만화), ([만화, 좋다], 애니)
# (context, target) : ([target index - 1, target index + 1], target)
#   -> (게임, 나), (게임, 만화), (만화, 게임), (만화, 애니), (애니, 만화), (애니, 좋다)
# (target, context[0]), (target, context[1])
skip_grams = []
for i in range(1, len(word_sequence) - 1): # 첫번째 마지막 문자는 target 아님
    # (context, target) : ([target index - 1, target index + 1], target)
    # 스킵그램을 만든 후, 저장은 단어의 고유 번호(index)로 저장합니다
    target = word_dict[word_sequence[i]]
    context = [word_dict[word_sequence[i - 1]], word_dict[word_sequence[i + 1]]]
    
    for w in context:
        skip_grams.append([target, w])

In [13]:
skip_grams

[[1, 2],
 [1, 8],
 [8, 1],
 [8, 2],
 [2, 8],
 [2, 1],
 [1, 2],
 [1, 6],
 [6, 1],
 [6, 2],
 [2, 6],
 [2, 1],
 [1, 2],
 [1, 7],
 [7, 1],
 [7, 6],
 [6, 7],
 [6, 8],
 [8, 6],
 [8, 7],
 [7, 8],
 [7, 3],
 [3, 7],
 [3, 8],
 [8, 3],
 [8, 6],
 [6, 8],
 [6, 1],
 [1, 6],
 [1, 8],
 [8, 1],
 [8, 4],
 [4, 8],
 [4, 5],
 [5, 4],
 [5, 1],
 [1, 5],
 [1, 6],
 [6, 1],
 [6, 4],
 [4, 6],
 [4, 11],
 [11, 4],
 [11, 5],
 [5, 11],
 [5, 1],
 [1, 5],
 [1, 6],
 [6, 1],
 [6, 8],
 [8, 6],
 [8, 9],
 [9, 8],
 [9, 1],
 [1, 9],
 [1, 2],
 [2, 1],
 [2, 3],
 [3, 2],
 [3, 1],
 [1, 3],
 [1, 3],
 [3, 1],
 [3, 2],
 [2, 3],
 [2, 11],
 [11, 2],
 [11, 3],
 [3, 11],
 [3, 2],
 [2, 3],
 [2, 12],
 [12, 2],
 [12, 14],
 [14, 12],
 [14, 13],
 [13, 14],
 [13, 1],
 [1, 13],
 [1, 2],
 [2, 1],
 [2, 0],
 [0, 2],
 [0, 10],
 [10, 0],
 [10, 15],
 [15, 10],
 [15, 1],
 [1, 15],
 [1, 8],
 [8, 1],
 [8, 6],
 [6, 8],
 [6, 11],
 [11, 6],
 [11, 6],
 [6, 11],
 [6, 8],
 [8, 6],
 [8, 1]]

In [14]:
#skip-gram 데이터에서 무작위로 데이터를 뽑아 입력값과 출력값의 배치 데이터를 생성하는 함수
def random_batch(data, size):
    random_inputs = []
    random_labels = []
    random_index = np.random.choice(range(len(data)), size, replace=True)
    
    for i in random_index:
        random_inputs.append(data[i][0]) # target
        random_iabels.append(data[i][1]) # context word
        
    return random_inputs, random_labels    

In [15]:
training_epoch = 300
learning_rate = 0.1
batch_size = 20
embedding_size = 2
num_sampled = 15
voc_size = len(word_list)

In [16]:
inputs = tf.placeholder(tf.float32, shape=[batch_size])

In [17]:
labels = tf.placeholder(tf.float32, shape=[])

<tf.Tensor 'Placeholder:0' shape=(20,) dtype=float32>

In [19]:
from tensorflow import keras

In [20]:
from keras.datasets import imdb

In [21]:
(train_data, train_labels),(test_data, test_labels) = imdb.load_data(num_words=10000)

17465344/17464789 [==============================] - 25s 1us/step


In [23]:
word_index = imdb.get_word_index()

1646592/1641221 [==============================] - 3s 2us/step


In [25]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

In [27]:
decoded_review = ' '.join([reverse_word_index.get(i-3,'?') for i in train_data[0]])

In [34]:
train_data

array([list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]),
       list([1, 194, 1153, 194, 8255, 78, 228,

In [35]:
def vectorize_sequence(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

In [36]:
x_train = vectorize_sequence(train_data)

In [37]:
x_train

array([[0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.]])

In [38]:
for i, seq in enumerate(train_data):
    if i < 1:
        print (i)
        print (seq)

0
[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
